In [0]:
# importing necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_extract, explode, split, year, round

In [0]:
# Creating SparkSession
spark = SparkSession.builder.appName("GFK MovieLens").getOrCreate()

In [0]:
# Movie information is in the file "movies.dat" and is in the following
# format:

# MovieID::Title::Genres

# - Titles are identical to titles provided by the IMDB (including
# year of release)
# - Genres are pipe-separated.

# 	* Action
# 	* Adventure
# 	* Animation
# 	* Children's
# 	* Comedy
# 	* Crime
# 	* Documentary
# 	* Drama
# 	* Fantasy
# 	* Film-Noir
# 	* Horror
# 	* Musical
# 	* Mystery
# 	* Romance
# 	* Sci-Fi
# 	* Thriller
# 	* War
# 	* Western

# - Some MovieIDs do not correspond to a movie due to accidental duplicate
# entries and/or test entries
# - Movies are mostly entered by hand, so errors and inconsistencies may exist

In [0]:
# Read movies file into pyspark dataframe seperating on "::" and providing schema
schema = "MovieID int, Title string, Genres string"
movies_df = spark.read.csv("/FileStore/tables/movies.dat",sep="::", schema = schema)
movies_df.show()

+-------+--------------------+--------------------+
|MovieID|               Title|              Genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Animation|Childre...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|        Comedy|Drama|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|Adventure|Children's|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Animation|Children's|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|      Drama|Thriller|
|     17|Sen

In [0]:
# User information is in the file "users.dat" and is in the following
# format:

# UserID::Gender::Age::Occupation::Zip-code

# All demographic information is provided voluntarily by the users and is
# not checked for accuracy.  Only users who have provided some demographic
# information are included in this data set.

# - Gender is denoted by a "M" for male and "F" for female
# - Age is chosen from the following ranges:

# 	*  1:  "Under 18"
# 	* 18:  "18-24"
# 	* 25:  "25-34"
# 	* 35:  "35-44"
# 	* 45:  "45-49"
# 	* 50:  "50-55"
# 	* 56:  "56+"

# - Occupation is chosen from the following choices:

# 	*  0:  "other" or not specified
# 	*  1:  "academic/educator"
# 	*  2:  "artist"
# 	*  3:  "clerical/admin"
# 	*  4:  "college/grad student"
# 	*  5:  "customer service"
# 	*  6:  "doctor/health care"
# 	*  7:  "executive/managerial"
# 	*  8:  "farmer"
# 	*  9:  "homemaker"
# 	* 10:  "K-12 student"
# 	* 11:  "lawyer"
# 	* 12:  "programmer"
# 	* 13:  "retired"
# 	* 14:  "sales/marketing"
# 	* 15:  "scientist"
# 	* 16:  "self-employed"
# 	* 17:  "technician/engineer"
# 	* 18:  "tradesman/craftsman"
# 	* 19:  "unemployed"
# 	* 20:  "writer"

In [0]:
# Read users file into pyspark dataframe seperating on "::" and providing schema
schema = "UserID int,Gender string, Age int, Occupation int, Zip_code int"
users_df = spark.read.csv("/FileStore/tables/users.dat", sep="::", schema = schema)
users_df.show()

+------+------+---+----------+--------+
|UserID|Gender|Age|Occupation|Zip_code|
+------+------+---+----------+--------+
|     1|     F|  1|        10|   48067|
|     2|     M| 56|        16|   70072|
|     3|     M| 25|        15|   55117|
|     4|     M| 45|         7|    2460|
|     5|     M| 25|        20|   55455|
|     6|     F| 50|         9|   55117|
|     7|     M| 35|         1|    6810|
|     8|     M| 25|        12|   11413|
|     9|     M| 25|        17|   61614|
|    10|     F| 35|         1|   95370|
|    11|     F| 25|         1|    4093|
|    12|     M| 25|        12|   32793|
|    13|     M| 45|         1|   93304|
|    14|     M| 35|         0|   60126|
|    15|     M| 25|         7|   22903|
|    16|     F| 35|         0|   20670|
|    17|     M| 50|         1|   95350|
|    18|     F| 18|         3|   95825|
|    19|     M|  1|        10|   48073|
|    20|     M| 25|        14|   55113|
+------+------+---+----------+--------+
only showing top 20 rows



In [0]:
# All ratings are contained in the file "ratings.dat" and are in the
# following format:

# UserID::MovieID::Rating::Timestamp

# - UserIDs range between 1 and 6040 
# - MovieIDs range between 1 and 3952
# - Ratings are made on a 5-star scale (whole-star ratings only)
# - Timestamp is represented in seconds since the epoch as returned by time(2)
# - Each user has at least 20 ratings


In [0]:
# Read ratings file into pyspark dataframe seperating on "::" and providing schema
schema = "UserID int,MovieID int, Rating int, Timestamp bigint"
ratings_df = spark.read.csv("/FileStore/tables/ratings.dat", sep="::", schema= schema)
ratings_df.show()

+------+-------+------+---------+
|UserID|MovieID|Rating|Timestamp|
+------+-------+------+---------+
|     1|   1193|     5|978300760|
|     1|    661|     3|978302109|
|     1|    914|     3|978301968|
|     1|   3408|     4|978300275|
|     1|   2355|     5|978824291|
|     1|   1197|     3|978302268|
|     1|   1287|     5|978302039|
|     1|   2804|     5|978300719|
|     1|    594|     4|978302268|
|     1|    919|     4|978301368|
|     1|    595|     5|978824268|
|     1|    938|     4|978301752|
|     1|   2398|     4|978302281|
|     1|   2918|     4|978302124|
|     1|   1035|     5|978301753|
|     1|   2791|     4|978302188|
|     1|   2687|     3|978824268|
|     1|   2018|     4|978301777|
|     1|   3105|     5|978301713|
|     1|   2797|     4|978302039|
+------+-------+------+---------+
only showing top 20 rows



In [0]:
# Please calculate the average ratings, per genre and year.
# By year we mean the year in which the movies were released.
# Please consider only movies, which were released after 1989.
# Please consider the ratings of all persons aged 18-49 years

In [0]:
# dropping unwanted columns from users_df

users_df = users_df.drop("Gender", "Occupation", "Zip_Code")
users_df.show()

+------+---+
|UserID|Age|
+------+---+
|     1|  1|
|     2| 56|
|     3| 25|
|     4| 45|
|     5| 25|
|     6| 50|
|     7| 35|
|     8| 25|
|     9| 25|
|    10| 35|
|    11| 25|
|    12| 25|
|    13| 45|
|    14| 35|
|    15| 25|
|    16| 35|
|    17| 50|
|    18| 18|
|    19|  1|
|    20| 25|
+------+---+
only showing top 20 rows



In [0]:
# dropping unwanted columns from ratings_df
ratings_df = ratings_df.drop("Timestamp")
ratings_df.show()

+------+-------+------+
|UserID|MovieID|Rating|
+------+-------+------+
|     1|   1193|     5|
|     1|    661|     3|
|     1|    914|     3|
|     1|   3408|     4|
|     1|   2355|     5|
|     1|   1197|     3|
|     1|   1287|     5|
|     1|   2804|     5|
|     1|    594|     4|
|     1|    919|     4|
|     1|    595|     5|
|     1|    938|     4|
|     1|   2398|     4|
|     1|   2918|     4|
|     1|   1035|     5|
|     1|   2791|     4|
|     1|   2687|     3|
|     1|   2018|     4|
|     1|   3105|     5|
|     1|   2797|     4|
+------+-------+------+
only showing top 20 rows



In [0]:
#In the movies_df dataframe, we will get year of release from the "Title" column using regexp_extract() function, taking the value between the final opening and closing brackets

movies_df = movies_df.withColumn("ReleaseYear",regexp_extract(col("Title"),r"\(([^()]+)\)$",1))
movies_df.show()

+-------+--------------------+--------------------+-----------+
|MovieID|               Title|              Genres|ReleaseYear|
+-------+--------------------+--------------------+-----------+
|      1|    Toy Story (1995)|Animation|Childre...|       1995|
|      2|      Jumanji (1995)|Adventure|Childre...|       1995|
|      3|Grumpier Old Men ...|      Comedy|Romance|       1995|
|      4|Waiting to Exhale...|        Comedy|Drama|       1995|
|      5|Father of the Bri...|              Comedy|       1995|
|      6|         Heat (1995)|Action|Crime|Thri...|       1995|
|      7|      Sabrina (1995)|      Comedy|Romance|       1995|
|      8| Tom and Huck (1995)|Adventure|Children's|       1995|
|      9| Sudden Death (1995)|              Action|       1995|
|     10|    GoldenEye (1995)|Action|Adventure|...|       1995|
|     11|American Presiden...|Comedy|Drama|Romance|       1995|
|     12|Dracula: Dead and...|       Comedy|Horror|       1995|
|     13|        Balto (1995)|Animation|

In [0]:
#In the movies_df dataframe, we need to seperate Genres column into individual genre for each movie, so using explode() function. 
# Now multiple rows are created for different genres.

movies_df = movies_df.withColumn("Genres",explode(split("Genres","[|]")))
movies_df.show()

+-------+--------------------+----------+-----------+
|MovieID|               Title|    Genres|ReleaseYear|
+-------+--------------------+----------+-----------+
|      1|    Toy Story (1995)| Animation|       1995|
|      1|    Toy Story (1995)|Children's|       1995|
|      1|    Toy Story (1995)|    Comedy|       1995|
|      2|      Jumanji (1995)| Adventure|       1995|
|      2|      Jumanji (1995)|Children's|       1995|
|      2|      Jumanji (1995)|   Fantasy|       1995|
|      3|Grumpier Old Men ...|    Comedy|       1995|
|      3|Grumpier Old Men ...|   Romance|       1995|
|      4|Waiting to Exhale...|    Comedy|       1995|
|      4|Waiting to Exhale...|     Drama|       1995|
|      5|Father of the Bri...|    Comedy|       1995|
|      6|         Heat (1995)|    Action|       1995|
|      6|         Heat (1995)|     Crime|       1995|
|      6|         Heat (1995)|  Thriller|       1995|
|      7|      Sabrina (1995)|    Comedy|       1995|
|      7|      Sabrina (1995

In [0]:
# Now we will finally join the 3 tables to get desired KPI. We keep movies_df as the left dataframe for the inner joins.
temp_df = movies_df.join(ratings_df, on=["MovieID"])
temp_df.show()

+-------+--------------------+----------+-----------+------+------+
|MovieID|               Title|    Genres|ReleaseYear|UserID|Rating|
+-------+--------------------+----------+-----------+------+------+
|   1193|One Flew Over the...|     Drama|       1975|     1|     5|
|    661|James and the Gia...|   Musical|       1996|     1|     3|
|    661|James and the Gia...|Children's|       1996|     1|     3|
|    661|James and the Gia...| Animation|       1996|     1|     3|
|    914| My Fair Lady (1964)|   Romance|       1964|     1|     3|
|    914| My Fair Lady (1964)|   Musical|       1964|     1|     3|
|   3408|Erin Brockovich (...|     Drama|       2000|     1|     4|
|   2355|Bug's Life, A (1998)|    Comedy|       1998|     1|     5|
|   2355|Bug's Life, A (1998)|Children's|       1998|     1|     5|
|   2355|Bug's Life, A (1998)| Animation|       1998|     1|     5|
|   1197|Princess Bride, T...|   Romance|       1987|     1|     3|
|   1197|Princess Bride, T...|    Comedy|       

In [0]:
final_df = temp_df.join(users_df, on=["UserID"])
final_df.show()

+------+-------+--------------------+----------+-----------+------+---+
|UserID|MovieID|               Title|    Genres|ReleaseYear|Rating|Age|
+------+-------+--------------------+----------+-----------+------+---+
|     1|   1193|One Flew Over the...|     Drama|       1975|     5|  1|
|     1|    661|James and the Gia...|   Musical|       1996|     3|  1|
|     1|    661|James and the Gia...|Children's|       1996|     3|  1|
|     1|    661|James and the Gia...| Animation|       1996|     3|  1|
|     1|    914| My Fair Lady (1964)|   Romance|       1964|     3|  1|
|     1|    914| My Fair Lady (1964)|   Musical|       1964|     3|  1|
|     1|   3408|Erin Brockovich (...|     Drama|       2000|     4|  1|
|     1|   2355|Bug's Life, A (1998)|    Comedy|       1998|     5|  1|
|     1|   2355|Bug's Life, A (1998)|Children's|       1998|     5|  1|
|     1|   2355|Bug's Life, A (1998)| Animation|       1998|     5|  1|
|     1|   1197|Princess Bride, T...|   Romance|       1987|    

In [0]:
# Convert Year column to int format so we can filter required data
final_df = final_df.withColumn("ReleaseYear",final_df.ReleaseYear.cast("int"))
final_df.printSchema()

root
 |-- UserID: integer (nullable = true)
 |-- MovieID: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genres: string (nullable = false)
 |-- ReleaseYear: integer (nullable = true)
 |-- Rating: integer (nullable = true)
 |-- Age: integer (nullable = true)



In [0]:
# Please calculate the average ratings, per genre and year.
# By year we mean the year in which the movies were released.
# Please consider only movies, which were released after 1989.
# Please consider the ratings of all persons aged 18-49 years

In [0]:
#First filtering rows where the release year is greater than 1989 and age  is between 18-49
#Then grouping the dataframe by "Genres", and "ReleaseYear" and calculating average rating


avg_rating = (final_df.filter("ReleaseYear>1989 and Age between 18 and 49").groupBy(["Genres","ReleaseYear"]).agg({"Rating":"avg"}))
display(avg_rating)

Genres,ReleaseYear,avg(Rating)
Horror,1990,3.1979010494752624
Documentary,1997,3.8688212927756656
Romance,1998,3.562386511024643
Children's,1992,3.259950248756219
Children's,1998,3.383314020857474
Mystery,1997,3.5251304586652017
Animation,1995,3.993894993894994
Adventure,1996,3.2408491947291362
Documentary,1996,4.103015075376884
Horror,1994,3.0639777468706537


In [0]:
#Rounding off the avg(Rating) column to nearest integer and using display() to create a visualization out of the table
avg_rating = avg_rating.select("*",round("avg(Rating)",0).alias("AvgRating")).drop("avg(Rating)")
display(avg_rating)

Genres,ReleaseYear,AvgRating
Horror,1990,3.0
Documentary,1997,4.0
Romance,1998,4.0
Children's,1992,3.0
Children's,1998,3.0
Mystery,1997,4.0
Animation,1995,4.0
Adventure,1996,3.0
Documentary,1996,4.0
Horror,1994,3.0


Databricks visualization. Run in Databricks to view.